In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division, print_function, unicode_literals

import sys
import sqlite3
import zlib
from pathlib import Path
import numpy as np

import msgpack
import pandas as pd

In [31]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
dir3 = dir1+'/spike_tools'
if not dir1 in sys.path: sys.path.append(dir1)
if not dir3 in sys.path: sys.path.append(dir3)

from spike_tools.utils.mworksutils import get_trial_indices

Loading lookup from /om/user/ssazaidi/miniconda3/envs/dicarlo_lab/lib/python3.7/site-packages/brainio_collection/lookup.csv


/om/user/ssazaidi/miniconda3/envs/dicarlo_lab/lib/python3.7/site-packages/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


# Reading mwk2

In [3]:
try:
    buffer
except NameError:
    # Python 3
    buffer = bytes

In [4]:
class MWK2Reader(object):

    _compressed_text_type_code = 1
    _compressed_msgpack_stream_type_code = 2

    def __init__(self, filename):
        self._conn = sqlite3.connect(filename)
        self._unpacker = msgpack.Unpacker(raw=False, strict_map_key=False)

    def close(self):
        self._conn.close()

    def __enter__(self):
        return self

    def __exit__(self, type, value, tb):
        self.close()

    @staticmethod
    def _decompress(data):
        return zlib.decompress(data, -15)

    def __iter__(self):
        for code, time, data in self._conn.execute('SELECT * FROM events'):
            if not isinstance(data, buffer):
                yield (code, time, data)
            else:
                try:
                    obj = msgpack.unpackb(data, raw=False)
                except msgpack.ExtraData:
                    # Multiple values, so not valid compressed data
                    pass
                else:
                    if isinstance(obj, msgpack.ExtType):
                        if obj.code == self._compressed_text_type_code:
                            yield (code,
                                   time,
                                   self._decompress(obj.data).decode('utf-8'))
                            continue
                        elif (obj.code ==
                              self._compressed_msgpack_stream_type_code):
                            data = self._decompress(obj.data)
                self._unpacker.feed(data)
                try:
                    while True:
                        yield (code, time, self._unpacker.unpack())
                except msgpack.OutOfData:
                    pass


### Dumping contents

In [6]:
filename = Path('/braintree/data2/active/users/ssazaidi/projects/stimulation-cIT/monkeys/oleo/mworksraw/datamaco-stim_corners-20211206-124257.mwk2')
assert filename.exists()

In [7]:
with MWK2Reader(filename) as event_file:
    code_to_name, name_to_code = {}, {}
    for code, time, data in event_file:
        if code == 0 and not code_to_name:
            code_to_name = dict((c, data[c]['tagname']) for c in data)
            name_to_code = dict((data[c]['tagname'], c) for c in data)
code_to_name

{4: '#allowAltFailover',
 41: 'eye_in_window',
 5: '#state_system_mode',
 46: 'reward_line',
 6: '#announceMessage',
 35: 'fixation_point_size_max',
 7: '#stimDisplayUpdate',
 48: 'mouse_x',
 8: '#experimentLoadProgress',
 45: 'stim_start_line',
 9: '#loadedExperiment',
 50: 'mouse_button_pressed',
 10: '#announceStimulus',
 55: 'key_c_pressed',
 11: '#announceSound',
 52: 'key_p_pressed',
 12: '#announceCalibrator',
 49: 'mouse_y',
 13: '#requestCalibrator',
 54: 'key_spacebar_pressed',
 14: '#announceCurrentState',
 43: 'experiment_state_line',
 15: '#announceTrial',
 56: 'stimulus_size',
 16: '#announceBlock',
 53: 'key_r_pressed',
 17: '#announceAssertion',
 58: 'stimulus_pos_y',
 18: '#serverName',
 63: 'stim_off_time',
 19: '#mainScreenInfo',
 60: 'ignore_time',
 20: '#warnOnSkippedRefresh',
 57: 'stimulus_pos_x',
 21: '#stopOnError',
 62: 'stim_on_time',
 22: '#realtimeComponents',
 51: 'key_x_pressed',
 23: 'eye_h_raw',
 64: 'stimulus_set_repetitions',
 24: 'eye_v_raw',
 61: 's

In [7]:
# codes = [
#             name_to_code['eye_h'],
#             name_to_code['eye_v'],
#             name_to_code['stim_key'],
#             name_to_code['RSVP_test_stim_index'],
#             name_to_code['stim_id'],
#             # Other meta data
#             name_to_code['stim_on_time'],
#             name_to_code['stim_off_time'],
#             name_to_code['stim_on_delay'],
#         ]

# data_dict = {
#     'code': [],
#     'name': [],
#     'time': [],
#     'data': [],
# }
# with MWK2Reader(filename) as event_file:
#     for code, time, data in event_file:
#         if code in codes:
#             data_dict['code'].append(code)
#             data_dict['name'].append(code_to_name[code])
#             data_dict['time'].append(time)
#             data_dict['data'].append(data)

In [21]:
name_to_code['stim_id']

80

In [8]:
codes = [
            name_to_code['eye_h'],
            name_to_code['eye_v'],
            name_to_code['trial_start_line'],
            name_to_code['correct_fixation'],
            name_to_code['stimulus_presented'],
    
            name_to_code['stim_id'],
            name_to_code['stim_current'],
            name_to_code['stim_pulse_period'],
            name_to_code['stim_num_pulses'],
    
            # Other meta data
            name_to_code['stim_on_time'],
            name_to_code['stim_off_time'],
            name_to_code['stim_on_delay'],
            name_to_code['stimulus_size'],
            name_to_code['fixation_window_size'],
            name_to_code['fixation_point_size_min'],
        ]

data_dict = {
    'code': [],
    'name': [],
    'time': [],
    'data': [],
}
with MWK2Reader(filename) as event_file:
    for code, time, data in event_file:
        if code in codes:
            data_dict['code'].append(code)
            data_dict['name'].append(code_to_name[code])
            data_dict['time'].append(time)
            data_dict['data'].append(data)

In [9]:
np.unique(data_dict['name'])

array(['correct_fixation', 'eye_h', 'eye_v', 'fixation_point_size_min',
       'fixation_window_size', 'stim_current', 'stim_id',
       'stim_num_pulses', 'stim_off_time', 'stim_on_delay',
       'stim_on_time', 'stim_pulse_period', 'stimulus_presented',
       'stimulus_size', 'trial_start_line'], dtype='<U23')

In [10]:
data_df = pd.DataFrame(data_dict)
data_df.head()

,code,name,time,data
0,28,eye_h,3258435836,0
1,28,eye_h,3258436891,0
2,28,eye_h,3259001779,2.767527
3,28,eye_h,3259001801,2.767527
4,28,eye_h,3259001822,2.769906


In [11]:
data_df.loc[data_df['name']=='stim_id']['data'].unique()

array(['', 'd-010', 'd-020', 'd-014', 'a-024', 'd-001', 'b-008', 'a-000',
       'a-019', 'd-007', 'a-010', 'a-022', 'a-028'], dtype=object)

In [12]:
data_df['name'].unique()

array(['eye_h', 'eye_v', 'fixation_window_size',
       'fixation_point_size_min', 'trial_start_line', 'stimulus_size',
       'stim_on_delay', 'stim_on_time', 'stim_off_time',
       'correct_fixation', 'stimulus_presented', 'stim_id',
       'stim_current', 'stim_pulse_period', 'stim_num_pulses'],
      dtype=object)

In [13]:
data_df = data_df.sort_values(by='time', ignore_index=True)
data_df.head()

,code,name,time,data
0,28,eye_h,3258435836,0
1,29,eye_v,3258435837,0
2,30,fixation_window_size,3258435838,2
3,34,fixation_point_size_min,3258435840,0.2
4,44,trial_start_line,3258435847,0


In [15]:
# data_df['first_in_trial'] = False  # Store whether the stimulus is first in a trial
# trial_start_df = data_df[(data_df.name == 'trial_start_line') | ((data_df.name == 'stimulus_presented') & (data_df.data != -1))]
# first_in_trial_times = [trial_start_df.time.values[i] for i in range(1, len(trial_start_df))
#                         if ((trial_start_df.name.values[i - 1] == 'trial_start_line') and
#                             (trial_start_df.name.values[i] == 'stimulus_presented'))]
data_df['first_in_trial'] = data_df['time'].apply(lambda x: True if x in first_in_trial_times else False)
data_df

KeyboardInterrupt: 

In [ ]:
stimulus_presented_df = data_df[data_df.name == 'stimulus_presented'].reset_index(drop=True)
correct_fixation_df = data_df[data_df.name == 'correct_fixation'].reset_index(drop=True)
correct_fixation_df = correct_fixation_df[stimulus_presented_df.data != -1].reset_index(drop=True)
stimulus_presented_df = stimulus_presented_df[stimulus_presented_df.data != -1].reset_index(drop=True)
correct_fixation_df['first_in_trial'] = stimulus_presented_df['first_in_trial']

In [ ]:
print(stimulus_presented_df.shape, correct_fixation_df.shape)

In [19]:
stimulus_presented_df

,code,name,time,data,first_in_trial
0,69,stimulus_presented,93357674072,1.0,True
1,69,stimulus_presented,93358008064,2.0,False
2,69,stimulus_presented,93358341230,3.0,False
3,69,stimulus_presented,93358675073,4.0,False
4,69,stimulus_presented,93359009477,5.0,False
...,...,...,...,...,...
1801,69,stimulus_presented,103434891928,35.0,False
1802,69,stimulus_presented,103435212335,36.0,False
1803,69,stimulus_presented,103435546301,37.0,False
1804,69,stimulus_presented,103435878946,38.0,False


In [20]:
output = {
        'stim_on_time_ms': data_df[data_df.name == 'stim_on_time']['data'].values[-1] / 1000.,
        'stim_off_time_ms': data_df[data_df.name == 'stim_off_time']['data'].values[-1] / 1000.,
        'stim_on_delay_ms': data_df[data_df.name == 'stim_on_delay']['data'].values[-1] / 1000.,
        'stimulus_size_degrees': data_df[data_df.name == 'stimulus_size']['data'].values[-1],
        'fixation_window_size_degrees': data_df[data_df.name == 'fixation_window_size']['data'].values[-1],
        'fixation_point_size_degrees': data_df[data_df.name == 'fixation_point_size_min']['data'].values[-1],
    }

In [287]:
eyeh, eyev = [], []
for t in stimulus_presented_df.time.values:
    eyeh.append(data_df[(data_df.name == 'eye_h') & (data_df.time >= t) & (data_df.time <= (t + output['stim_on_time_ms'] * 1000.))].data.values.tolist())
    eyev.append(data_df[(data_df.name == 'eye_v') & (data_df.time >= t) & (data_df.time <= (t + output['stim_on_time_ms'] * 1000.))].data.values.tolist())
print(len(eyeh), len(eyev))

393 393


In [288]:
threshold = output['stim_on_time_ms'] // 2

for i in range(len(eyeh)):
    if correct_fixation_df.iloc[i]['data'] == -1: # Skip if marked incorrect
        continue
    
    if len(eyeh[i]) < threshold or len(eyev[i]) < threshold:
        correct_fixation_df.at[i, 'data'] = -1
    elif np.any([np.abs(_) > 2 for _ in eyeh[i]]) or np.any([np.abs(_) > 2 for _ in eyev[i]]):
        correct_fixation_df.at[i, 'data'] = -1

In [289]:
stimulus_presented_df[314:340]

,code,name,time,data,first_in_trial
314,67,stimulus_presented,191731782615,24.0,True
315,67,stimulus_presented,191732015671,25.0,False
316,67,stimulus_presented,191732249123,23.0,False
317,67,stimulus_presented,191732482517,4.0,False
318,67,stimulus_presented,191732715726,15.0,False
319,67,stimulus_presented,191772716711,18.0,True
320,67,stimulus_presented,191772950378,20.0,False
321,67,stimulus_presented,191782851559,14.0,True
322,67,stimulus_presented,191783085201,16.0,False
323,67,stimulus_presented,191783318249,1.0,False


In [290]:
correct_fixation_df[314:340]

,code,name,time,data,first_in_trial
314,66,correct_fixation,191731997581,1.0,True
315,66,correct_fixation,191732230733,-1.0,False
316,66,correct_fixation,191732464126,-1.0,False
317,66,correct_fixation,191732697547,-1.0,False
318,66,correct_fixation,191732830858,-1.0,False
319,66,correct_fixation,191772932129,-1.0,True
320,66,correct_fixation,191772980225,-1.0,False
321,66,correct_fixation,191783066707,-1.0,True
322,66,correct_fixation,191783299530,-1.0,False
323,66,correct_fixation,191783533328,-1.0,False


In [25]:
# def get_trial_indices(events, df = False):
#     if df:
#         times = np.array([row.time for i, row in events.iterrows()])
#     else:
#         times = np.array([i.time for i in events])
# #     print(len(times))
#     diff_times = np.diff(times)
# #     print(len(diff_times))

#     trials = []

#     mini_trial = [0]

#     for i, t in enumerate(diff_times):
#         if t < 1e7:
#             mini_trial.append(i+1)
#         else:
#             trials.append(mini_trial)
#             mini_trial = [i+1]
#     trials.append(mini_trial)
#     print(i)
#     return trials

# current_events = data_df.loc[data_df['name'] == 'stim_current']
# id_events = data_df.loc[data_df['name'] == 'stim_id']
# current_times = np.array([row.time for i, row in current_events.iterrows()])
# id_times = np.array([row.time for i, row in id_events.iterrows()])
# current_trials = get_trial_indices(current_events, df=True)
# id_trials = get_trial_indices(id_events, df=True)

# correct_id = []
# correct_current = []
# for i in range(len(id_trials)):
# #     print((current_times[current_trials[i][0]] - id_times[id_trials[i][0]]), (current_times[current_trials[i+1][0]] - id_times[id_trials[i][0]]))
# #     print((current_times[current_trials[i][-1]] - id_times[id_trials[i][-1]]), (current_times[current_trials[i+1][-1]] - id_times[id_trials[i][-1]]))

    
#     current_trial = current_trials[i]
#     id_trial = id_trials[i]
#     if '' in np.array(id_events.iloc[id_trial].data):
#         continue

#         #     print([current_events.data.iloc[j] for j in current_trial], [id_events.data.iloc[j] for j in id_trial])

#     for idx, j in enumerate(id_trial):
#         print(idx, current_trial[idx], j)
#         correct_current.append(current_trial[idx])
#         correct_id.append(j)
        
        

In [29]:
# data = data_df
from utils.mworksutils import get_trial_indices

ModuleNotFoundError: No module named 'utils'

In [124]:
current_events = data.loc[data['name'] == 'stim_current'].reset_index(drop=True)
id_events = data.loc[data['name'] == 'stim_id'].reset_index(drop=True)
current_times = np.array([row.time for i, row in current_events.iterrows()])
id_times = np.array([row.time for i, row in id_events.iterrows()])

current_trials = get_trial_indices(current_events, df=True, delay_sec=1)
id_trials = get_trial_indices(id_events, df=True, delay_sec = 0.5)


correct_id = []
correct_current = []

curr_idx = 0

while np.all(np.array(current_events.iloc[current_trials[curr_idx]].data) == 1):
    curr_idx += 1

for i in range(len(id_trials)):
    current_trial = current_trials[curr_idx]
    id_trial = id_trials[i]

    if '' in np.array(id_events.iloc[id_trial].data):
        continue

    if len(id_trial) > 8:
        id_trial = id_trial[-8:]


    for idx, j in enumerate(id_trial):
        try:
            correct_current.append(current_trial[idx])
            correct_id.append(j)
        except:
            pass

    if len(id_trial) > 1:
        curr_idx += 1

[    1033   566422 22078004 ...   200860   202091 10045259]
3981 501
[    1033   566395 22078031 ...   331711   333833  3223306]
3771 533


In [125]:
len(correct_current), len(correct_id)

(3768, 3768)

In [126]:
len(current_trials), len(id_trials)

(501, 533)

In [127]:
start_ = 810
end_ = 850
print(correct_id[start_:end_])
list(zip(np.arange(start_,end_), correct_id[start_:end_], correct_current[start_:end_], id_events.iloc[correct_id[start_:end_]].data.values, current_events.iloc[correct_current[start_:end_]].data.values))

[815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854]


[(810, 815, 863, 'a-028', 0),
 (811, 816, 870, 'a-028', 0),
 (812, 817, 870, 'a-028', 0),
 (813, 818, 871, 'd-020', 64),
 (814, 819, 872, 'd-001', 4),
 (815, 820, 873, 'd-014', 0),
 (816, 821, 878, 'd-014', 0),
 (817, 822, 879, 'a-010', 32),
 (818, 823, 880, 'a-024', 32),
 (819, 824, 881, 'b-008', 64),
 (820, 825, 882, 'a-022', 48),
 (821, 826, 883, 'a-000', 4),
 (822, 827, 884, 'a-019', 48),
 (823, 828, 885, 'd-007', 16),
 (824, 829, 894, 'd-010', 16),
 (825, 830, 895, 'd-020', 0),
 (826, 831, 896, 'd-014', 64),
 (827, 832, 897, 'a-024', 4),
 (828, 833, 898, 'd-001', 0),
 (829, 834, 899, 'b-008', 32),
 (830, 835, 900, 'a-000', 64),
 (831, 836, 901, 'a-019', 32),
 (832, 837, 902, 'd-007', 48),
 (833, 838, 903, 'a-010', 4),
 (834, 839, 904, 'a-022', 16),
 (835, 840, 905, 'a-028', 48),
 (836, 841, 906, 'd-014', 48),
 (837, 842, 907, 'd-001', 64),
 (838, 843, 908, 'b-008', 0),
 (839, 844, 909, 'a-024', 64),
 (840, 845, 910, 'a-019', 16),
 (841, 846, 911, 'd-007', 32),
 (842, 847, 912, 'a-

In [106]:
# list(zip(id_trials[120:140], id_times[120:140]))
prev_trial_time = id_times[id_trials[119][0]]
for i in range(121,130):
    print(id_trials[i], id_events.data.values[id_trials[i]], id_times[id_trials[i][0]])
    time_diff = id_times[id_trials[i][0]] - prev_trial_time
    prev_trial_time = id_times[id_trials[i][0]]
    print("{:e}".format(time_diff))

[812] ['d-010'] 9152682818
5.234220e+08
[813] ['d-010'] 9163613825
1.093101e+07
[814, 815] ['d-010' 'a-028'] 9166500829
2.887004e+06
[816] ['a-028'] 9179401999
1.290117e+07
[817, 818, 819, 820] ['a-028' 'd-020' 'd-001' 'd-014'] 9183372404
3.970405e+06
[821, 822, 823, 824, 825, 826, 827, 828] ['d-014' 'a-010' 'a-024' 'b-008' 'a-022' 'a-000' 'a-019' 'd-007'] 9194478802
1.110640e+07
[829, 830, 831, 832, 833, 834, 835, 836] ['d-010' 'd-020' 'd-014' 'a-024' 'd-001' 'b-008' 'a-000' 'a-019'] 9205269818
1.079102e+07
[837, 838, 839, 840, 841, 842, 843, 844] ['d-007' 'a-010' 'a-022' 'a-028' 'd-014' 'd-001' 'b-008' 'a-024'] 9216161850
1.089203e+07
[845, 846, 847, 848, 849, 850, 851, 852] ['a-019' 'd-007' 'a-000' 'a-022' 'a-028' 'd-020' 'a-010' 'd-010'] 9227045819
1.088397e+07


In [102]:
correct_current[800]

855

In [120]:
# list(zip(id_trials[120:140], id_times[120:140]))
prev_trial_time = current_times[current_trials[119][0]]
for i in range(110,120):
    print(current_trials[i], current_events.data.values[current_trials[i]], current_times[current_trials[i][0]])
    time_diff = current_times[current_trials[i][0]] - current_times[current_trials[i-1][-1]]
    prev_trial_time = current_times[current_trials[i][0]]
    print("{:e}".format(time_diff))

[846, 847, 848, 849, 850, 851, 852, 853] [32 32 32 64 0 64 64 32] 8621679794
7.547731e+06
[854, 855, 856, 857, 858, 859, 860, 861] [32 48 16 4 48 0 48 32] 8634076652
1.103614e+07
[862, 863, 864, 865, 866, 867, 868, 869] [64 0 64 4 0 32 32 64] 8647030107
1.159330e+07
[870, 871, 872, 873, 874, 875, 876, 877] [0 64 4 0 32 32 64 48] 9169047017
5.207037e+08
[878, 879, 880, 881, 882, 883, 884, 885] [0 32 32 64 48 4 48 16] 9186527149
1.617293e+07
[886, 887, 888, 889, 890, 891, 892, 893] [0 32 32 64 48 4 48 16] 9188593420
7.048100e+05
[894, 895, 896, 897, 898, 899, 900, 901] [16 0 64 4 0 32 64 32] 9199309622
9.358067e+06
[902, 903, 904, 905, 906, 907, 908, 909] [48 4 16 48 48 64 0 64] 9210114353
9.497194e+06
[910, 911, 912, 913, 914, 915, 916, 917] [16 32 48 0 16 16 16 4] 9220989620
9.514816e+06
[918, 919, 920, 921, 922, 923, 924, 925] [16 0 64 48 64 4 16 0] 9231872206
9.524873e+06


In [123]:
0.8e6 - 7.048100e+05

95190.0

In [110]:
from itertools import product

currents = [0, 4,16, 32, 48, 64]


# all_channels = np.array(np.meshgrid(channel_letters, channel_numbers))
# all_channels = np.core.defchararray.add(np.core.defchararray.add(all_channels[0],'-'),np.char.zfill(all_channels[1], 3))
# all_channels = all_channels.flatten()



#### FOR ACTUAL  CORNER STIMULATION
### PIT Array
# all_channels_unique = ['a-000', 'd-008',  'd-018', 'a-016', 'd-000', 
#                      'a-024', 'd-016', 'a-031', 'd-025', 'a-022', 'd-005',  
#                                                              'd-010', 'a-028']

## CIT ARRAY
all_channels_unique = ['d-001', 'd-014',  'd-020', 'a-024', 'a-000', 'a-028', 
                                'd-010', 'b-008', 'a-022', 'a-010', 'a-019', 'd-007']


#### FOR STIM DEMO

# all_channels_unique = ['a-000', 'b-008',  'b-018', 'a-016', 'b-000', 
#                      'a-024', 'b-016', 'a-031', 'b-025', 'a-022', 'b-005', 'b-031', 
#                                                             'b-000', 'b-010', 'a-028']

all_channels = list(product(all_channels_unique, currents))
#
# all_refs = ['a-008', 'b-008', 'a-006', 'b-020', 'a-009', 'b-026', 'b-022']
#
#  # Update this list based on the electrodes you want to stimulate (Use LGA to Headstage map)
#
# np.random.seed(1)
# np.random.shuffle(all_channels)

np.random.seed(1)
all_channel_ids = np.array([np.random.permutation(all_channels_unique) for i in range(len(currents))]).flatten()
i = 0
chan = all_channel_ids[i]
num_good_ones = 0
back_list = all_channel_ids[i - 8:None]

while chan not in back_list or i < len(all_channels):
    if chan in back_list:
        temp = np.copy(chan)
        swap_num = 1
        while all_channel_ids[i + swap_num] in back_list:
            swap_num += 1
        all_channel_ids[i] = all_channel_ids[i + swap_num]
        all_channel_ids[i + swap_num] = temp
        i = 0
        num_good_ones = 0
        chan = all_channel_ids[i]
    else:
        i = (i + 1) % len(all_channel_ids)
        num_good_ones += 1
        try:
            chan = all_channel_ids[i]
        except:
            break
    if i < 8:
        back_list = all_channel_ids[i - 8:None]
    else:
        back_list = all_channel_ids[i - 8:i]

    if chan not in back_list and num_good_ones > 205:
        break

all_currents = np.zeros(all_channel_ids.shape)
for channel in all_channels_unique:
    idxs = np.where(all_channel_ids == channel)
    all_currents[idxs] = np.random.permutation(currents)
all_channels = [(i, j) for i, j in zip(list(all_channel_ids), list(all_currents.astype(int)))]


In [119]:
all_channels

[('d-010', 16),
 ('d-020', 0),
 ('d-014', 64),
 ('a-024', 4),
 ('d-001', 0),
 ('b-008', 32),
 ('a-000', 64),
 ('a-019', 32),
 ('d-007', 48),
 ('a-010', 4),
 ('a-022', 16),
 ('a-028', 48),
 ('d-014', 48),
 ('d-001', 64),
 ('b-008', 0),
 ('a-024', 64),
 ('a-019', 16),
 ('d-007', 32),
 ('a-000', 48),
 ('a-022', 0),
 ('a-028', 16),
 ('d-020', 16),
 ('a-010', 16),
 ('d-010', 4),
 ('a-024', 16),
 ('a-019', 0),
 ('d-007', 64),
 ('d-001', 48),
 ('a-022', 64),
 ('a-028', 4),
 ('d-014', 16),
 ('a-010', 0),
 ('d-010', 48),
 ('b-008', 16),
 ('a-000', 16),
 ('d-020', 4),
 ('a-024', 48),
 ('a-022', 4),
 ('a-028', 32),
 ('d-007', 4),
 ('a-019', 4),
 ('d-010', 0),
 ('b-008', 4),
 ('a-000', 0),
 ('d-020', 32),
 ('d-001', 32),
 ('d-014', 32),
 ('a-010', 64),
 ('d-007', 0),
 ('a-019', 64),
 ('a-028', 64),
 ('d-010', 32),
 ('a-000', 32),
 ('d-020', 48),
 ('d-001', 16),
 ('d-014', 4),
 ('a-010', 48),
 ('a-024', 0),
 ('b-008', 48),
 ('a-022', 32),
 ('d-010', 64),
 ('a-028', 0),
 ('d-020', 64),
 ('d-001', 4)

In [17]:
stim_id_df = id_events.iloc[correct_id]
stim_current_df = current_events.iloc[correct_current]


In [96]:
len(correct_id)

3768

In [20]:
stim_current_df

,code,name,time,data,first_in_trial
147732,81,stim_current,3371198241,48,False
147885,81,stim_current,3371399192,16,False
148290,81,stim_current,3371600977,16,False
148693,81,stim_current,3371802601,4,False
149098,81,stim_current,3372004100,48,False
...,...,...,...,...,...
13065233,81,stim_current,17633750353,0,False
13065240,81,stim_current,17633903253,16,False
13065475,81,stim_current,17634104394,16,False
13065878,81,stim_current,17634306409,16,False


In [21]:
id_events

,code,name,time,data,first_in_trial
11,80,stim_id,3258435893,,False
26,80,stim_id,3258436926,,False
93,80,stim_id,3259003321,,False
44150,80,stim_id,3281081352,,False
44223,80,stim_id,3318393363,,False
...,...,...,...,...,...
14105017,80,stim_id,18256103543,d-020,False
14105690,80,stim_id,18256438664,d-014,False
14106357,80,stim_id,18256770375,a-024,False
14107028,80,stim_id,18257104208,d-001,False


In [22]:
current_events

,code,name,time,data,first_in_trial
12,81,stim_current,3258435894,1,False
27,81,stim_current,3258436927,1,False
94,81,stim_current,3259003349,1,False
44151,81,stim_current,3281081353,1,False
44224,81,stim_current,3318393402,1,False
...,...,...,...,...,...
14093525,81,stim_current,18249728251,0,False
14093530,81,stim_current,18249879306,64,False
14093613,81,stim_current,18250080166,4,False
14094018,81,stim_current,18250282257,0,False
